In [1]:
import os
import sys
sys.path.append(os.path.abspath('../src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use('ggplot')


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)


In [2]:
def fetch_data(ticker, start, end):
    import yfinance as yf
    df = yf.download(ticker, start=start, end=end)
    df.dropna(inplace=True)
    return df


In [3]:
from data_loader import fetch_data

start_date = "2020-01-01"
end_date = "2024-01-01"

df = fetch_data('AAPL', start_date, end_date)
df.head()


ModuleNotFoundError: No module named 'yfinance'